# Interactive Plotting with Jupyter

There are several ways to interactively plot. In this tutorial I will show how to interact with 2D and 1D data.  There are other ways to interact with large tables of data using either [Bokeh](https://docs.bokeh.org/en/latest/index.html) (shown the Skyfit notebook) or [Glue](http://docs.glueviz.org/en/stable). A non-python based solution that also works with large tables of data is Topcat. 

Most of the methods here will work on the command line.  In order to make this work within Jupyter you will need the following modules.

```
conda install -c conda-forge ipympl

conda install -c conda-forge ipywidgets
```

https://ipywidgets.readthedocs.io/


In [ ]:
import glob,os,sys

import numpy as np
import matplotlib.pyplot as plt 

import astropy.io.fits as pyfits
import astropy.units as u
from astroquery.skyview import SkyView

import ipywidgets as widgets

Here we need an image to play with, we can either download it via SkyView or load one from our machine.

In [ ]:
ext = 0

# download an image
pflist = SkyView.get_images(position='M82', survey=['SDSSr'], radius=10 * u.arcmin)
pf = pflist[0] # first element of the list, might need a loop if multiple images

# or load an image
#pf = pyfits.open('m82.fits')

image = pf[ext].data

Next we need to turn on the interactive plotting.  

In [ ]:
# turn-on interactive plots
%matplotlib widget 

# Display an image (2D data)

We plot a 2D image using imshow, we can set the scale of the image as well as the colormap.

In [ ]:
#plt.ioff()
fig = plt.figure(figsize=[6,6])
plt.ion()

p = fig.add_subplot(111)
p.imshow(image, interpolation='Nearest', origin='lower', vmin=-10, vmax=20, cmap='viridis')

plt.show()

# Add an event to the display

There are several types of matplotlib events that you can use to interact with a figure. 

A few useful events are the following:

`button_press_event` 	
`button_release_event` 	
`key_press_event`  
`key_release_event`  

For more information on event handling and examples check out the following website: 
https://matplotlib.org/stable/users/event_handling.html

Here we add a python function linking to link to the `key_press_event`. The function checks for the which key being pressed and if the condition is met runs its code, in this case plotting a red point on the image.  We can easily add more keys adding more functionaly to our interactive figure.

In [ ]:
#plt.ioff()
fig = plt.figure(figsize=[6,6])
plt.ion()

p = fig.add_subplot(111)
p.imshow(image, interpolation='Nearest', origin='lower', vmin=-10, vmax=20, cmap='viridis')

def on_key_press(event):
    xc, yc = event.xdata, event.ydata

    if event.key == 'm':
        p.plot(xc,yc,'ro', markersize=5)
        fig.canvas.draw_idle

fig.canvas.mpl_connect('key_press_event', on_key_press)

plt.show()

# Add output to the display with the event

If we want to display the coordinate of the points we mark, we need to use the Output widget.

In [ ]:
#plt.ioff()
fig = plt.figure(figsize=[6,6])
plt.ion()

p = fig.add_subplot(111)
p.imshow(image, interpolation='Nearest', origin='lower', vmin=-10, vmax=20, cmap='viridis')

out = widgets.Output()
@out.capture()
def on_key_press(event):
    xc, yc = event.xdata, event.ydata

    if event.key == 'm':
        p.plot(xc,yc,'ro', markersize=5)
        fig.canvas.draw_idle
        
        print("[%.1f, %.1f] = %.4f" % (xc, yc, image[int(yc),int(xc)]))

fig.canvas.mpl_connect('key_press_event', on_key_press)

display(out)


We can also write a Python class, this makes it more convient for dealing with multiple interactive events (i.e. keypress, mouse clicking, dragging, etc).

In [ ]:
class GUI_inter:

    def __init__(self,fig,img):
        self.fig = fig
        self.p = self.fig.gca()

        self.img = img

        self.display()

    def display(self,sigma=10.0):
        plt.clf()

        self.v0 = np.mean(self.img) - sigma * np.std(self.img)
        self.v1 = np.mean(self.img) + sigma * np.std(self.img)

        self.p = self.fig.add_subplot(111)
        self.p.imshow(self.img, interpolation='Nearest', origin='lower', 
                      vmin=self.v0, vmax=self.v1, cmap='viridis')
        plt.draw()

    def on_key_press(self, event):
        xc, yc = event.xdata, event.ydata

        if event.key == 'm':
            self.p.plot(xc,yc,'ro', markersize=5)
            fig.canvas.draw_idle

            print("[%.2f, %.2f]" % (xc,yc))

In [ ]:
fig = plt.figure(figsize=[6,6])
G = GUI_inter(fig, image)
fig.canvas.mpl_connect('key_press_event', G.on_key_press)
#display(fig)

# Interactive 1D data

In [ ]:
slice = image[150,:]

fig = plt.figure(figsize=[6,6])
p = fig.add_subplot(111)
p.plot(slice)
plt.show()

In [ ]:
zl,xl = image.shape

fig = plt.figure(figsize=[6,6])
p = fig.add_subplot(111)
#p.set_yscale('log')

slice = image[150,:]
line, = p.plot(slice)

def update(change):
    line.set_ydata(image[change.new,:])
    fig.canvas.draw()

int_slider = widgets.IntSlider(
    value=150,
    min=0,
    max=zl,
    step=1,
    description='Z-axis:',
    continuous_update=False
)
int_slider.observe(update, 'value')
int_slider

In [ ]:
from astroquery.sdss import SDSS
from astropy import coordinates

In [ ]:
ra, dec = 148.969687, 69.679383

In [ ]:
co = coordinates.SkyCoord(ra=ra, dec=dec,unit=(u.deg, u.deg), frame='fk5')

In [ ]:
xid = SDSS.query_region(co, radius=10 * u.arcmin, spectro=True)
sp = SDSS.get_spectra(matches=xid)

In [ ]:
pf = sp[0]
ext = 1
pf[ext].header
tab = pf[ext].data

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.plot(10**tab['loglam'],tab['flux'])
ax.set_xlabel('Wavelength [Angstroms]')
ax.set_ylabel('Flux')

In [ ]:
# turn-off interactive plots
%matplotlib inline 

# Resources

https://ipywidgets.readthedocs.io/

https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html

https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html

https://kapernikov.com/ipywidgets-with-matplotlib/

https://matplotlib.org/stable/users/event_handling.html

https://docs.bokeh.org/en/latest/index.html

http://docs.glueviz.org/en/stable